<a href="https://colab.research.google.com/github/jainstuti/youtube-views-prediction/blob/main/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
!cp './drive/MyDrive/DeepLearning/yt_views_prediction/prepared_data.csv' 'data.csv'

In [ ]:
df = pd.read_csv('data.csv', index_col=0)

In [ ]:
df.head()

,publishedAt,nTags,categoryId,duration,definition,caption,embeddable,publicStatsViewable,madeForKids,viewCount,likeCount,dislikeCount,commentCount,country,channelPublishedAt,channelViewCount,subscriberCount,hiddenSubscriberCount,videoCount,defaultTab,title_polarity_scores,description_polarity_scores,ct_polarity_scores
0,7.851640e+07,27,26,28827,0,0,1,1,0,11156226,29940,6355,3792,43,1.980226e+08,54890686,204000,0,229,0,0.00,-0.8328,0.4404
1,2.398782e+07,22,24,135,1,0,1,1,0,5976911,132234,1120,1251,21,4.356119e+07,306619353,508000,0,128,0,-0.34,-0.9995,-0.2732
2,5.269903e+07,6,10,114,1,0,1,1,0,5295431,95076,1506,709,21,6.349829e+07,6130973,10600,0,53,0,0.00,0.0000,0.0000
3,2.922970e+08,8,10,230,0,1,1,1,0,8331680,39605,1835,705,21,2.923203e+08,37044354,34600,0,54,0,0.00,-0.1511,0.0000
4,3.581129e+07,16,10,133,1,0,1,1,0,2917687,90590,447,544,21,6.632913e+07,23156819,41300,0,600,0,0.00,-0.3400,0.0000


## Split the data into train & test sets

In [ ]:
# Try removing some columns
# cols_to_remove = ['publishedAt', 'channelPublishedAt']
# df.drop(cols_to_remove, axis=1, inplace=True)

In [ ]:
train_perc = 0.9

In [ ]:
train = df.iloc[:int(train_perc * df.shape[0])]
train.shape

(2697, 21)

In [ ]:
test = df.iloc[int(train_perc * df.shape[0]):]
test.shape

(300, 21)

In [ ]:
train_x = train.drop('viewCount', axis=1)
train_y = train['viewCount']
print(train_x.shape)
print(train_y.shape)

(2697, 20)
(2697,)


In [ ]:
test_x = test.drop('viewCount', axis=1)
test_y = test['viewCount']
print(test_x.shape)
print(test_y.shape)

(300, 20)
(300,)


## Function to compare accuracies

In [ ]:
def findRMSE(y, y_pred):
  return int(np.sqrt(sum((y - y_pred) ** 2)))

## Apply Linear Regression

In [ ]:
!pip3 install -U scikit-learn

     |████████████████████████████████| 22.3MB 1.2MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression(normalize=True, positive=True)

In [ ]:
reg.fit(train_x.values, train_y.values)

LinearRegression(normalize=True, positive=True)

In [ ]:
reg.score(train_x.values, train_y.values)

0.9607624028130562

In [ ]:
reg.score(test_x.values, test_y.values)

0.7434343066471245

In [ ]:
reg.coef_

array([0.00000000e+00, 1.09536476e+05, 1.65066330e+01, 0.00000000e+00,
       5.97699898e+05, 0.00000000e+00, 0.00000000e+00, 6.84749562e+06,
       1.60258722e+02, 2.90215759e+02, 1.18891065e+02, 0.00000000e+00,
       4.08378522e-05, 0.00000000e+00, 3.98374475e+04, 1.34851921e+01,
       9.28297015e+05, 1.31464381e+06, 9.99293961e+05, 0.00000000e+00])

In [ ]:
pred_y = reg.predict(test_x.values)

In [ ]:
idx = 0
a = int(pred_y[idx])
b = int(test_y.values[idx])
print(f'{a}, {b}')
print((100 * (a - b)) / b)

46572849, 58562518
-20.473281220592327


In [ ]:
for i in range(len(pred_y)):
  a = int(pred_y[i])
  b = int(test_y.values[i])
  if b != 0:
    print(f'{(100 * (a - b)) / b} %, Predicted: {a}, Actual: {b}')
  else:
    print(f'{int(a)}, {b} has 0 views')
  print()

-20.473281220592327 %, Predicted: 46572849, Actual: 58562518

301.2616242015994 %, Predicted: 862556, Actual: 214961

811.6316652489344 %, Predicted: 5776472, Actual: 633641

-250.3905152553801 %, Predicted: -455005, Actual: 302549

115.29647029999636 %, Predicted: 650275, Actual: 302037

216.83520820770858 %, Predicted: 1098154, Actual: 346601

-1427.6553132106678 %, Predicted: -2054998, Actual: 154784

295.3812983885646 %, Predicted: 2259272, Actual: 571416

222.69709039998122 %, Predicted: 455140531, Actual: 141042651

-1939.1797305187847 %, Predicted: -2488355, Actual: 135297

-50702.52237591538 %, Predicted: -3109525, Actual: 6145

-38.25152375880229 %, Predicted: 250437, Actual: 405576

210.92926647410073 %, Predicted: 496240, Actual: 159599

-1271.422439343458 %, Predicted: -1146190, Actual: 97846

-13525.479641867405 %, Predicted: -2519157, Actual: 18764

-236.26668067668382 %, Predicted: -1374661, Actual: 1008802

-4698.304219981471 %, Predicted: -3424771, Actual: 74479

-239.

In [ ]:
test_y.values[0]

58562518

In [ ]:
print(f'Root mean squared error = {findRMSE(test_y, pred_y)}')

Root mean squared error = 425519503


## Try KNN

In [ ]:
# KNN algorithm
def dist(x1, x2):
  return np.sqrt(sum((x1 - x2) ** 2))


def knn(X, Y, querypoint, k=5):
  vals = []
  
  m = X.shape[0]
  for i in range(m):
    d = dist(querypoint, X[i])
    vals.append((d, Y[i]))

  vals = sorted(vals)

  # Find nearest k points
  vals = vals[:k]

  vals = np.array(vals)

  return int(np.mean(vals[:, 1]))

In [ ]:
INDEX = 0
print(knn(train_x.values, train_y.values, test_x.values[INDEX]))
print(test_y.values[INDEX])

472427898
58562518


In [ ]:
pred_y_knn = []
for i in range(len(pred_y)):
  a = knn(train_x.values, train_y.values, test_x.values[INDEX], 20)
  b = int(test_y.values[i])
  pred_y_knn.append(a)
  if b != 0:
    print(f'{(100 * (a - b)) / b} %, Predicted: {a}, Actual: {b}')
  else:
    print(f'{int(a)}, {b} has 0 views')
  print()

128.9224295307794 %, Predicted: 134062739, Actual: 58562518

62266.075241555445 %, Predicted: 134062739, Actual: 214961

21057.522792874828 %, Predicted: 134062739, Actual: 633641

44211.08316338841 %, Predicted: 134062739, Actual: 302549

44286.197386412925 %, Predicted: 134062739, Actual: 302037

38579.2706887747 %, Predicted: 134062739, Actual: 346601

86512.78878953897 %, Predicted: 134062739, Actual: 154784

23361.495477900513 %, Predicted: 134062739, Actual: 571416

-4.948795240668016 %, Predicted: 134062739, Actual: 141042651

98987.73956554839 %, Predicted: 134062739, Actual: 135297

2181555.6387306754 %, Predicted: 134062739, Actual: 6145

32954.89945164408 %, Predicted: 134062739, Actual: 405576

83899.73621388605 %, Predicted: 134062739, Actual: 159599

136914.02101261166 %, Predicted: 134062739, Actual: 97846

714367.8053719889 %, Predicted: 134062739, Actual: 18764

13189.301468474487 %, Predicted: 134062739, Actual: 1008802

179900.72369392714 %, Predicted: 134062739, Act

In [ ]:
print(f'Root mean squared error = {findRMSE(test_y, pred_y_knn)}')

Root mean squared error = 2356038611


## Try DNN using tensorflow